In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
from keras.models import load_model
import pickle

In [77]:
# Load dataset test
test_data = pd.read_csv("destinasi_wisata.csv")
test_data

,Unnamed: 0,User_Id,Place_Id,Place_Ratings,Place_Name,Description,City,Category,city_category
0,0,1,179,3,Candi Ratu Boko,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Yogyakarta,Budaya,Yogyakarta Budaya
1,1,1,344,2,Pantai Marina,"Pantai Marina (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ​ꦩꦫꦶꦤ, trans...",Semarang,Bahari,Semarang Bahari
2,2,1,5,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Jakarta,Taman Hiburan,Jakarta Taman Hiburan
3,3,1,373,3,Museum Kereta Ambarawa,Museum Kereta Api Ambarawa (bahasa Inggris: In...,Semarang,Budaya,Semarang Budaya
4,4,1,101,4,Kampung Wisata Sosro Menduran,Kampung wisata Sosromenduran merupakan kampung...,Yogyakarta,Budaya,Yogyakarta Budaya
...,...,...,...,...,...,...,...,...,...
9995,9995,300,425,2,Waterpark Kenjeran Surabaya,Waterpark Kenjeran Surabaya merupakan wisata k...,Surabaya,Taman Hiburan,Surabaya Taman Hiburan
9996,9996,300,64,4,Museum Sasmita Loka Ahmad Yani,Museum Sasmita Loka Ahmad Yani adalah salah sa...,Jakarta,Budaya,Jakarta Budaya
9997,9997,300,311,3,The Lodge Maribaya,The Lodge Maribaya adalah salah satu tempat wi...,Bandung,Cagar Alam,Bandung Cagar Alam
9998,9998,300,279,4,Masjid Agung Trans Studio Bandung,Masjid Agung Trans Studio Bandung (TSB) berdir...,Bandung,Tempat Ibadah,Bandung Tempat Ibadah


In [79]:
# Load model
model = load_model("wisata_model.h5")
model

ValueError: ignored

In [80]:
# Prepare dataframe
place_df = test_data[['Place_Id','Place_Name','Category','Place_Ratings']]
place_df.columns = ['id','place_name','category','rating']

In [81]:
def dict_encoder(col, data):
    unique_val = data[col].unique().tolist()
    val_to_val_encoded = {x: i for i, x in enumerate(unique_val)}
    val_encoded_to_val = {i: x for i, x in enumerate(unique_val)}
    return val_to_val_encoded, val_encoded_to_val

In [82]:
# Encoding User_Id
user_to_user_encoded, user_encoded_to_user = dict_encoder('User_Id', test_data)
test_data['user'] = test_data['User_Id'].map(user_to_user_encoded)

In [83]:
# Encoding Place_Id
place_to_place_encoded, place_encoded_to_place = dict_encoder('Place_Id', test_data)
test_data['place'] = test_data['Place_Id'].map(place_to_place_encoded)

In [84]:
# Choose a random user
user_id = test_data.User_Id.sample(1).iloc[0]
place_visited_by_user = test_data[test_data.User_Id == user_id]

In [85]:
# Create data for places not visited by the user
place_not_visited = place_df[~place_df['id'].isin(place_visited_by_user.Place_Id.values)]['id']
place_not_visited = list(set(place_not_visited).intersection(set(place_to_place_encoded.keys())))
place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_place_array = np.hstack(([[user_encoder]] * len(place_not_visited), place_not_visited))

In [87]:
# Recommendation function
def recommend_places(recommended_place_ids, num_recommendations=7):
    # Predict ratings using the model
    ratings = model.predict(user_place_array).flatten()

    # Sort the places based on the highest ratings
    sorted_indices = np.argsort(ratings, axis=0)[-7:][::-1][:num_recommendations]

    # Get the recommended places information
    recommended_places = [
        place_encoded_to_place.get(place_not_visited[x][0]) for x in sorted_indices
    ]

    return recommended_places

print('Daftar rekomendasi untuk: {}'.format('User ' + str(user_id)))
print('===' * 15,'\n')
print('----' * 15)
print('Tempat dengan rating wisata paling tinggi dari user')
print('----' * 15)

top_place_user = (
    place_visited_by_user.sort_values(by='Place_Ratings', ascending=False)
    .head(5)
    .Place_Id.values
)

place_df_rows = place_df[place_df['id'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.place_name, ':', row.category)

print('')
print('----' * 15)
print('Top 7 place recommendation')
print('----' * 15)

recommended_place = place_df[place_df['id'].isin(top_place_user)]
for row, i in zip(recommended_place.itertuples(), range(1,8)):
    print(i,'.', row.place_name, '\n    ', row.category, ',', 'Rating Wisata ', row.rating,'\n')

print('==='*15)

Daftar rekomendasi untuk: User 199

------------------------------------------------------------
Tempat dengan rating wisata paling tinggi dari user
------------------------------------------------------------
Hutan Wisata Tinjomoyo Semarang : Cagar Alam
Hutan Wisata Tinjomoyo Semarang : Cagar Alam
Sumur Gumuling : Taman Hiburan
Heha Sky View : Taman Hiburan
Taman Situ Lembang : Taman Hiburan
Sumur Gumuling : Taman Hiburan
Taman Keputran : Taman Hiburan
Heha Sky View : Taman Hiburan
Sumur Gumuling : Taman Hiburan
Hutan Wisata Tinjomoyo Semarang : Cagar Alam
Hutan Wisata Tinjomoyo Semarang : Cagar Alam
Taman Keputran : Taman Hiburan
Hutan Wisata Tinjomoyo Semarang : Cagar Alam
Taman Keputran : Taman Hiburan
Sumur Gumuling : Taman Hiburan
Sumur Gumuling : Taman Hiburan
Taman Keputran : Taman Hiburan
Heha Sky View : Taman Hiburan
Taman Keputran : Taman Hiburan
Taman Situ Lembang : Taman Hiburan
Heha Sky View : Taman Hiburan
Heha Sky View : Taman Hiburan
Taman Situ Lembang : Taman Hiburan
